In [1]:
include("../src/Julia.jl")

parseData (generic function with 1 method)

In [2]:
pwd()

"/Users/serbanstan/git/TreePCG/julia/testing"

In [3]:
gpath = "../../graphs/";

In [4]:
graphs = readdir("../../graphs");

In [5]:
for cur in graphs
    path = gpath * cur * "/"
    
    try
        a = readFromFile(path * "graph.mtx");
        x = readFromFile(path * "x.vec");
        
        tree = []
        for ind in 1:5
            tr = readFromFile(path * "tree$(ind).mtx");
            
            u,v,w = findnz(tr)
            for i in 1:length(w)
                w[i] = a[u[i],v[i]]
            end
            tr = sparse(u,v,w)
            
            push!(tree, tr);
        end
        
        la = lap(a);
        x = x - mean(x)
        b = la * x; b = b - mean(b);
        
        numIts = 1000;
        @time for i in 4:5
            @time F = treeSolver(tree[i])
            @time F(b);

            # a run on standard Float64 data types
            @time myx64,dbg = pcgV(la, b, F, x, maxits=numIts, verbose=false);
            norm(lap(a) * myx64 - b) / norm(b)
            insert!(dbg, 1, "$(numIts) iterations")
            insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
            writeToFile(path * "_log_julia_tree$(i)_default.txt", dbg)

            # now useing higer precision data types
            for hp in [64,128,256,512,1024]
                set_bigfloat_precision(hp)
                laHP = toHighPrecision(la; precision=hp)
                treeHP = toHighPrecision(tree[i]; precision=hp)
                xHP = toHighPrecision(x; precision=hp);
                bHP = toHighPrecision(b; precision=hp);
                FHP = treeSolver(treeHP)

                @time myxHP,dbg = pcgV(laHP, bHP, FHP, xHP, maxits=numIts, verbose=false);
                insert!(dbg, 1, "$(numIts) iterations")
                insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
                writeToFile(path * "_log_julia_tree$(i)_$(hp).txt", dbg)
            end
        end
            
        println("finished running on ", cur, "\n")
    catch
        println("skipping ", cur, "\n")
    end
end

  0.419816 seconds (652.03 k allocations: 30.341 MB, 0.88% gc time)
  0.135832 seconds (290.67 k allocations: 7.535 MB)
  7.378864 seconds (198.13 M allocations: 4.675 GB, 6.35% gc time)
400.901203 seconds (2.38 G allocations: 60.475 GB, 33.77% gc time)
433.673867 seconds (2.38 G allocations: 66.255 GB, 34.74% gc time)
484.296715 seconds (2.38 G allocations: 77.833 GB, 33.37% gc time)
543.527960 seconds (2.38 G allocations: 100.988 GB, 33.15% gc time)
623.412680 seconds (2.38 G allocations: 147.299 GB, 31.86% gc time)
  0.016020 seconds (50.18 k allocations: 5.763 MB)
  0.005665 seconds (197.90 k allocations: 3.631 MB)
  8.789767 seconds (198.17 M allocations: 4.669 GB, 23.24% gc time)
405.138414 seconds (2.38 G allocations: 60.470 GB, 34.15% gc time)
437.378688 seconds (2.38 G allocations: 66.259 GB, 34.52% gc time)
495.382164 seconds (2.38 G allocations: 77.836 GB, 32.84% gc time)
539.216795 seconds (2.38 G allocations: 100.992 GB, 33.31% gc time)
620.822407 seconds (2.38 G allocatio